# Ko→En QE (CPU, VS Code, local, `- b.txt` pairing)
- `X.txt`(ko) ↔ `X - b.txt`(en)
- COMET Kiwi(gated) → public wmt21 automatic fallback
- CPU-only execution

In [1]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


## 0) (Optional) Install

In [2]:
# %pip install pandas==2.2.2 pyarrow>=14,<20 scikit-learn==1.5.2 sentencepiece evaluate==0.4.2
# %pip install transformers==4.44.2 datasets==2.21.0 accelerate==0.34.2 unbabel-comet==2.2.3
# %pip install torch==2.8.0 --index-url https://download.pytorch.org/whl/cpu
print("Hello World!")

Hello World!


## 1) Paths

In [3]:
import os, re, glob, json
DATA_DIR = "./data"
OUT_DIR  = "./qe_runs"
os.makedirs(OUT_DIR, exist_ok=True)
print("DATA_DIR =", os.path.abspath(DATA_DIR))
print("OUT_DIR  =", os.path.abspath(OUT_DIR))


DATA_DIR = c:\Users\pockg\OneDrive\Desktop\Hojumoney\DEV\v14\making_model\data
OUT_DIR  = c:\Users\pockg\OneDrive\Desktop\Hojumoney\DEV\v14\making_model\qe_runs


## 2) Find pairs (`X.txt` ↔ `X - b.txt`)

In [4]:
def find_pairs(data_dir):
    txts = glob.glob(os.path.join(data_dir, "*.txt"))
    base_map = {}
    for p in txts:
        name = os.path.basename(p)
        m = re.match(r"^(.*)\.txt$", name)
        if not m: 
            continue
        stem = m.group(1)
        if stem.endswith(" b"):
            continue
        ko_path = p
        en_path = os.path.join(data_dir, f"{stem} b.txt")
        if os.path.exists(en_path):
            base_map[stem] = (ko_path, en_path)
    return base_map

pairs = find_pairs(DATA_DIR)
print("Found pairs:", len(pairs))
for k, v in list(pairs.items())[:10]:
    print(k, "->", v)


Found pairs: 21
Accommodation - Easy -> ('./data\\Accommodation - Easy.txt', './data\\Accommodation - Easy b.txt')
Accommodation - Hard -> ('./data\\Accommodation - Hard.txt', './data\\Accommodation - Hard b.txt')
Accommodation - Medium -> ('./data\\Accommodation - Medium.txt', './data\\Accommodation - Medium b.txt')
Bank - Easy -> ('./data\\Bank - Easy.txt', './data\\Bank - Easy b.txt')
Bank - Hard -> ('./data\\Bank - Hard.txt', './data\\Bank - Hard b.txt')
Bank - Medium -> ('./data\\Bank - Medium.txt', './data\\Bank - Medium b.txt')
Culture - Easy -> ('./data\\Culture - Easy.txt', './data\\Culture - Easy b.txt')
Culture - Hard -> ('./data\\Culture - Hard.txt', './data\\Culture - Hard b.txt')
Culture - Medium -> ('./data\\Culture - Medium.txt', './data\\Culture - Medium b.txt')
Developer - Easy -> ('./data\\Developer - Easy.txt', './data\\Developer - Easy b.txt')


## 3) Load & align

In [5]:
def load_lines(path):
    with open(path, "r", encoding="utf-8-sig") as f:
        lines = [ln.strip() for ln in f.readlines()]
    return lines

def build_examples(pair_map):
    rows = []
    for stem, (ko_path, en_path) in pair_map.items():
        ko_lines = load_lines(ko_path)
        en_lines = load_lines(en_path)
        n = min(len(ko_lines), len(en_lines))
        if len(ko_lines) != len(en_lines):
            print(f"[WARN] mismatch '{stem}': ko={len(ko_lines)} en={len(en_lines)} -> {n}")
        for i in range(n):
            ko = ko_lines[i]; en = en_lines[i]
            if not ko or not en: 
                continue
            rows.append({"file": stem, "idx": i, "ko": ko, "en": en})
    return rows

raw_rows = build_examples(pairs)
print("Total aligned rows:", len(raw_rows))
raw_rows[:3]


Total aligned rows: 21000


[{'file': 'Accommodation - Easy',
  'idx': 0,
  'ko': '게스트하우스에 냉장고가 있나요?',
  'en': 'Is there a refrigerator in the guesthouse?'},
 {'file': 'Accommodation - Easy',
  'idx': 1,
  'ko': '레지던스에서 짐 보관 가능할까요?',
  'en': "Do you think it'll be possible to keep your things in Regance?"},
 {'file': 'Accommodation - Easy',
  'idx': 2,
  'ko': '모텔까지 셔틀로 얼마나 걸리나요?',
  'en': 'How long will it take to get to the motel?'}]

## 4) COMET QE scoring (CPU) — Kiwi→wmt21 fallback

In [7]:
# === COMET scoring (robust fallback, no internal API) ===
import torch, pandas as pd
from comet import download_model, load_from_checkpoint

# 1) Candidates: Open QE first → (last) Kiwi (Gated: HF login/approval required)
CANDIDATES = [
    "wmt20-comet-qe-da", # ✅ Open QE (requires src+mt only)
    "wmt21-comet-qe-da", #PublicQE (not depending on environment)
    "wmt22-cometkiwi-da", # 🔒 gated (Hugging Face login + Access consent required)
]

selected_id, qe_model = None, None
last_err = None

for mid in CANDIDATES:
    try:
        print(f"[COMET] Trying: {mid}")
        mpath = download_model(mid)
        model = load_from_checkpoint(mpath)
        model.eval()
        selected_id, qe_model = mid, model
        print(f"[COMET] Loaded: {mid}")
        break
    except Exception as e:
        print(f"[COMET] Fail: {mid} -> {e}")
        last_err = e

if qe_model is None:
    raise RuntimeError(
        "I haven't received the COMET model available. Please copy the error log above and send it to me."
        "To write Kiwi, you need access consent from the model page after 'huggingface-cli login'."
    ) from last_err

is_qe = ("qe" in selected_id) or ("kiwi" in selected_id)

# 2) Placement scoring (QE: src+mt/DA: src+mt+ref required)
batch_size = 32
scores = []
for i in range(0, len(raw_rows), batch_size):
    batch = raw_rows[i:i+batch_size]
    if is_qe:
        data = [{"src": r["ko"], "mt": r["en"]} for r in batch]
    else:
        # DA models require a reference (ref). If there is no ref, the meaning falls.
        data = [{"src": r["ko"], "mt": r["en"], "ref": r["en"]} for r in batch]

    with torch.no_grad():
        out = qe_model.predict(data, batch_size=min(16, len(data)), gpus=0)  # CPU

    for s in out["scores"]:
        # Rescale COMET score (-1..1) to 0..100
        score_0_100 = float(max(0.0, min(1.0, (s + 1) / 2.0))) * 100.0
        scores.append(score_0_100)

df = pd.DataFrame(raw_rows)
df["score"] = scores
display({"model": selected_id})
display(df.head()); display(df["score"].describe())

c:\Users\pockg\OneDrive\Desktop\Hojumoney\DEV\v14\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[COMET] Trying: wmt20-comet-qe-da


wmt20-comet-qe-da is already in cache.
Lightning automatically upgraded your loaded checkpoint from v1.3.5 to v2.5.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint C:\Users\pockg\.cache\torch\unbabel_comet\wmt20-comet-qe-da\checkpoints\model.ckpt`
c:\Users\pockg\OneDrive\Desktop\Hojumoney\DEV\v14\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Encoder model frozen.
c:\Users\pockg\OneDrive\Desktop\Hojumoney\DEV\v14\.venv\Lib\site-packages\pytorch_lightning\core\saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']
💡 Tip: For seamle

[COMET] Loaded: wmt20-comet-qe-da


Predicting DataLoader 0: 100%|██████████| 2/2 [00:02<00:00,  1.39s/it]
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|██████████| 2/2 [00:02<00:00,  1.01s/it]
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|██████████| 2/2 [00:02<00:00,  1.12s/it]
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs

{'model': 'wmt20-comet-qe-da'}

,file,idx,ko,en,score
0,Accommodation - Easy,0,게스트하우스에 냉장고가 있나요?,Is there a refrigerator in the guesthouse?,97.722170
1,Accommodation - Easy,1,레지던스에서 짐 보관 가능할까요?,Do you think it'll be possible to keep your th...,50.002044
2,Accommodation - Easy,2,모텔까지 셔틀로 얼마나 걸리나요?,How long will it take to get to the motel?,50.592725
3,Accommodation - Easy,3,패밀리룸은 있나요?,Is there a family room?,85.914856
4,Accommodation - Easy,4,조식은 포함되어 있나요?,Do you include the eclipse?,50.002301


count    21000.000000
mean        62.736878
std         13.393798
min         50.002017
25%         50.003369
50%         59.942552
75%         70.215014
max         99.010512
Name: score, dtype: float64

## 5) Split & save parquet

In [8]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)
train_path = os.path.join(OUT_DIR, "train.parquet")
val_path   = os.path.join(OUT_DIR, "val.parquet")
train_df.to_parquet(train_path, index=False)
val_df.to_parquet(val_path, index=False)
print(train_path, val_path)


./qe_runs\train.parquet ./qe_runs\val.parquet


## 6) Tokenize (XLM-R)

In [ ]:
# === XLM-R QE (Regression) — load parquet → tokenize → train ===
import os, numpy as np
from datasets import load_dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
)

# 0) Paths
OUT_DIR = globals().get("OUT_DIR", "./qe_runs")
os.makedirs(OUT_DIR, exist_ok=True)
train_path = os.path.join(OUT_DIR, "train.parquet")
val_path   = os.path.join(OUT_DIR, "val.parquet")

# 1) Load data (expects columns: 'ko','en','score' at minimum)
data = load_dataset(
    "parquet",
    data_files={"train": train_path, "validation": val_path},
)

# 2) Build input text: [KO] ... [EN] ...
def build_text(batch):
    ko = batch.get("ko", [""]*len(batch[next(iter(batch))]))
    en = batch.get("en", [""]*len(batch[next(iter(batch))]))
    return {"text": [f"[KO] {k} [EN] {e}" for k, e in zip(ko, en)]}

data = data.map(build_text, batched=True, desc="Build input text [KO]/[EN]")

# 3) Scale labels: score 50~100 → labels 0~1
def map_labels(batch):
    if "labels" in batch:
        return {}
    s = np.array(batch["score"], dtype=float)
    return {"labels": ((s - 50.0) / 50.0).astype("float32")}

data = data.map(map_labels, batched=True, desc="Scale labels 50~100 → 0~1")

# 4) Tokenizer / Model
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
model = AutoModelForSequenceClassification.from_pretrained("xlm-roberta-base", num_labels=1)
model.config.problem_type = "regression"

# 5) Tokenize
def tok(batch):
    return tokenizer(
        batch["text"],
        truncation=True,
        max_length=256,
    )

data = data.map(tok, batched=True, remove_columns=[c for c in data["train"].column_names if c not in ("labels") and c not in ("input_ids","attention_mask")],
                desc="Tokenize")

# 6) Metrics
def compute_metrics(eval_pred):
    preds, labels = eval_pred
    if isinstance(preds, tuple):
        preds = preds[0]
    preds = preds.reshape(-1)
    labels = np.array(labels).reshape(-1)
    mae = float(np.mean(np.abs(preds - labels)))
    rmse = float(np.sqrt(np.mean((preds - labels) ** 2)))
    pearson = 0.0 if (preds.std() == 0 or labels.std() == 0) else float(np.corrcoef(preds, labels)[0, 1])
    return {"mae": mae, "rmse": rmse, "pearson": pearson}

# 7) Training args (CPU-friendly)
args = TrainingArguments(
    output_dir=os.path.join(OUT_DIR, "xlmr_qe"),
    learning_rate=1e-5,
    weight_decay=0.01,
    warmup_ratio=0.06,
    max_grad_norm=1.0,
    gradient_accumulation_steps=2,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=200,
    save_strategy="steps",
    save_steps=200,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="pearson",
    greater_is_better=True,
    dataloader_num_workers=4,
    dataloader_pin_memory=False,
    remove_unused_columns=False,
    report_to="none",
    seed=42,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=data["train"],
    eval_dataset=data["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5, early_stopping_threshold=5e-4)],
)

trainer.train()
save_dir = os.path.join(OUT_DIR, "xlmr_qe", "best")
trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)
print("Saved to:", save_dir)


PyTorch version 2.8.0 available.
Generating train split: 18900 examples [00:00, 1244796.03 examples/s]
Generating validation split: 2100 examples [00:00, 906736.50 examples/s]
Scale labels 50~100 → 0~1: 100%|██████████| 2100/2100 [00:00<00:00, 97641.43 examples/s]
c:\Users\pockg\OneDrive\Desktop\Hojumoney\DEV\v14\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream 

{'loss': 0.095, 'grad_norm': 2.3956806659698486, 'learning_rate': 2.347417840375587e-06, 'epoch': 0.04}


  3%|▎         | 100/3543 [08:05<4:15:50,  4.46s/it]

{'loss': 0.0811, 'grad_norm': 1.6544015407562256, 'learning_rate': 4.694835680751174e-06, 'epoch': 0.08}


  4%|▍         | 150/3543 [11:55<4:17:50,  4.56s/it]

...

{'loss': 0.0061, 'grad_norm': 0.4236988127231598, 'learning_rate': 2.792792792792793e-07, 'epoch': 2.92}


 99%|█████████▉| 3500/3543 [5:05:36<03:31,  4.91s/it]

{'loss': 0.0061, 'grad_norm': 0.2775803506374359, 'learning_rate': 1.2912912912912912e-07, 'epoch': 2.96}


100%|██████████| 3543/3543 [5:09:00<00:00,  5.23s/it]


{'train_runtime': 18540.3684, 'train_samples_per_second': 3.058, 'train_steps_per_second': 0.191, 'train_loss': 0.016756799727081075, 'epoch': 3.0}
Saved to: ./qe_runs\xlmr_qe\best


## 7) Train XLM-R regression (CPU)

In [ ]:
import os
import numpy as np
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
)

# 모델/토크나이저 (이미 로드돼 있으면 이 두 줄은 생략해도 됨)
tokenizer = tokenizer if "tokenizer" in globals() else AutoTokenizer.from_pretrained("xlm-roberta-base")
model = AutoModelForSequenceClassification.from_pretrained(
    "xlm-roberta-base",
    num_labels=1,
)

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    # HF가 (logits,) 형태로 줄 때를 대비
    if isinstance(preds, tuple):
        preds = preds[0]
    preds = preds.reshape(-1)
    labels = np.array(labels).reshape(-1)
    mae = float(np.mean(np.abs(preds - labels)))
    rmse = float(np.sqrt(np.mean((preds - labels) ** 2)))
    # 분산 0 방지
    pearson = 0.0 if (preds.std() == 0 or labels.std() == 0) else float(np.corrcoef(preds, labels)[0, 1])
    return {"mae": mae, "rmse": rmse, "pearson": pearson}

args = TrainingArguments(
    output_dir=os.path.join(OUT_DIR, "xlmr_qe"),
    # 안정화 & 속도 관련 핵심 튜닝
    learning_rate=1e-5,
    weight_decay=0.01,
    warmup_ratio=0.06,
    max_grad_norm=1.0,
    gradient_accumulation_steps=2,

    # 배치/에폭
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    num_train_epochs=3,

    # 로깅/평가/체크포인트
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=200,
    save_strategy="steps",
    save_steps=200,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="pearson",
    greater_is_better=True,

    # CPU 환경 가정 (GPU 없으면 pin_memory 경고 회피)
    dataloader_num_workers=4,
    dataloader_pin_memory=False,

    # 기타
    report_to="none",
    seed=42,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5, early_stopping_threshold=5e-4)],
)

# 빠른 맛보기만 하고 싶으면 아래 주석 해제
# trainer.train(resume_from_checkpoint=False, max_steps=400)

trainer.train()
trainer.save_model(os.path.join(OUT_DIR, "xlmr_qe/best"))
tokenizer.save_pretrained(os.path.join(OUT_DIR, "xlmr_qe/best"))


c:\Users\pockg\OneDrive\Desktop\Hojumoney\DEV\v14\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


NameError: name 'OUT_DIR' is not defined

## 8) Inference & batch scoring

In [ ]:
import torch, csv
from transformers import AutoTokenizer, AutoModelForSequenceClassification

CKPT = os.path.join(OUT_DIR, "xlmr_qe/best")
tok = AutoTokenizer.from_pretrained(CKPT)
mdl = AutoModelForSequenceClassification.from_pretrained(CKPT)
mdl.eval()

def score_translation(ko, en):
    text = f"{ko} </s> {en}"
    enc = tok(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
    with torch.no_grad():
        out = mdl(**enc).logits.squeeze().item()
    return float(max(0.0, min(100.0, out)))

def score_file_pair(ko_path, en_path, out_csv):
    def load_lines(path):
        with open(path, "r", encoding="utf-8-sig") as f:
            lines = [ln.strip() for ln in f.readlines()]
        return lines
    ko_lines = load_lines(ko_path)
    en_lines = load_lines(en_path)
    n = min(len(ko_lines), len(en_lines))
    with open(out_csv, "w", newline="", encoding="utf-8") as f:
        w = csv.writer(f)
        w.writerow(["idx","ko","en","score"])
        for i in range(n):
            s = score_translation(ko_lines[i], en_lines[i])
            w.writerow([i, ko_lines[i], en_lines[i], f"{s:.2f}"])

print (score_translation ("Please fix the code now", "Please fix the code now"))
# Example of use:
# stem, (ko_p, en_p) = next(iter(pairs.items()))
# out_csv = os.path.join(OUT_DIR, f"{stem}_scores.csv")
# score_file_pair(ko_p, en_p, out_csv)
# print("Saved:", out_csv)


c:\Users\pockg\OneDrive\Desktop\Hojumoney\DEV\v14\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'os' is not defined